In [27]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, roc_auc_score
from skopt import BayesSearchCV
from skopt.space import Real
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
import warnings
from sklearn.model_selection import cross_val_score, StratifiedKFold
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, roc_auc_score

SVC con kernel "linear" + Hiperparámetro "c"

In [28]:
df = pd.read_csv('brain_tumor_dataset.csv')
df=df.head(100)
df['Gender'] = df['Gender'].map({'Male':0,'Female':1})
df['Radiation_Treatment'] = df['Radiation_Treatment'].map({'No':0,'Yes':1})
df['Surgery_Performed'] = df['Surgery_Performed'].map({'No':0,'Yes':1})
df['Chemotherapy'] = df['Chemotherapy'].map({'No':0,'Yes':1})
df['Family_History'] = df['Family_History'].map({'No':0,'Yes':1})
df['MRI_Result'] = df['MRI_Result'].map({'Negative':0,'Positive':1})
df['Follow_Up_Required'] = df['Follow_Up_Required'].map({'No':0,'Yes':1})

In [29]:
X=df.drop(columns=["Tumor_Type","Patient_ID"])
X=pd.get_dummies(X, columns=['Location','Histology','Stage','Symptom_1','Symptom_2','Symptom_3'])
y = df["Tumor_Type"]

In [ ]:
# --- Preprocesamiento ---
numerical_transformer = StandardScaler()
numerical_features = X.columns
preprocessor = ColumnTransformer(
    transformers=[('num', numerical_transformer, numerical_features)]
)

# --- Modelo base ---
svc = SVC(kernel='linear', probability=True)
          
# --- Pipeline --
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', svc)
])

# --- Validación cruzada ---
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

# --- Espacio de búsqueda (C continuo) ---
search_spaces = {
    'model__C': Real(1e-3, 1e3, prior='log-uniform')
}

# --- Métrica AUC ---
scorer = make_scorer(roc_auc_score, needs_proba=True, multi_class='ovr')

# --- Búsqueda bayesiana ---
opt = BayesSearchCV(
    estimator=pipeline,
    search_spaces=search_spaces,
    scoring=scorer,
    cv=cv,
    n_iter=10,          # iteraciones
    n_jobs=1,
    random_state=42
)

# --- Entrenar ---
opt.fit(X, y)

# --- Resultados ---
print("Mejor valor de C:", opt.best_params_['model__C'])
print("AUC promedio en CV:", opt.best_score_)